In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Load data set

In [3]:
train_transforms = transforms.Compose([
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomCrop(size=(28, 28), padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # mean and std of MNIST dataset
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # mean and std of MNIST dataset
])

train_dataset = datasets.MNIST(root='./../data', train=True, transform=train_transforms, download=True)
test_dataset = datasets.MNIST(root='./../data', train=False, transform=test_transforms, download=True)

batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f'Number of training batches: {len(train_loader)}')
print(f'Number of testing batches: {len(test_loader)}')

Number of training batches: 600
Number of testing batches: 100


# Train test functions

In [4]:
def train(model, criterion, optimizer, epochs, scheduler=None):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if (i + 1) % 100 == 0:
                print(f'Epoch {epoch + 1}/{epochs}, Step {i + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')

        if scheduler is not None:
            scheduler.step()

        # Epoch summary
        train_accuracy = correct / total * 100
        test_accuracy = test(model)
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {test_accuracy:.2f}%\n')


def test(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    return accuracy


# Model summary

In [5]:
def print_model_summary(model):
    def layer_summary(layer):
        output_shape = None
        if hasattr(layer, 'out_channels'):
            output_shape = (layer.out_channels, "H_out", "W_out")
        elif hasattr(layer, 'out_features'):
            output_shape = (layer.out_features)
        elif isinstance(layer, torch.nn.modules.pooling._MaxPoolNd):
            output_shape = (layer.kernel_size, "H_out", "W_out")

        num_params = sum(p.numel() for p in layer.parameters() if p.requires_grad)
        return output_shape, num_params

    model_name = model.__class__.__name__
    print(f"'{model_name}' Model Summary:")

    print("="*75)
    print(f"{'Layer':<30} {'Output Shape':<30} {'Param #':<15}")
    print("="*75)
    
    for name, layer in model.named_children():
        output_shape, num_params = layer_summary(layer)
        print(f"{name:<30} {str(output_shape):<30} {num_params:<15}")
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print("="*75)
    print(f"Total params:          {total_params}")
    print(f"Trainable params:      {trainable_params}")
    print(f"Non-trainable params:  {total_params - trainable_params}")

# CNN

In [6]:
class AccuracyOptimCNN(nn.Module):
    def __init__(self):
        super(AccuracyOptimCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 7 * 7, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = torch.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)

        x = x.view(-1, 64 * 7 * 7) # Flatten the tensor
        x = self.dropout1(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x
    
ao_model = AccuracyOptimCNN().to(device)
ao_criterion = nn.CrossEntropyLoss()
ao_optimizer = optim.Adam(ao_model.parameters(), lr=0.001)
ao_scheduler = optim.lr_scheduler.StepLR(ao_optimizer, step_size=5, gamma=0.5)

print_model_summary(ao_model)

'AccuracyOptimCNN' Model Summary:
Layer                          Output Shape                   Param #        
conv1                          (32, 'H_out', 'W_out')         320            
conv2                          (64, 'H_out', 'W_out')         18496          
conv3                          (32, 'H_out', 'W_out')         18464          
conv4                          (64, 'H_out', 'W_out')         18496          
bn1                            None                           64             
bn2                            None                           128            
bn3                            None                           64             
bn4                            None                           128            
pool                           (2, 'H_out', 'W_out')          0              
dropout1                       None                           0              
fc1                            64                             200768         
dropout2                      

In [7]:
train(ao_model, ao_criterion, ao_optimizer, epochs=10, scheduler=ao_scheduler)

Epoch 1/10, Step 100/600, Loss: 1.0296
Epoch 1/10, Step 200/600, Loss: 0.5742
Epoch 1/10, Step 300/600, Loss: 0.4029
Epoch 1/10, Step 400/600, Loss: 0.5613
Epoch 1/10, Step 500/600, Loss: 0.4507
Epoch 1/10, Step 600/600, Loss: 0.4192
Epoch 1/10, Loss: 0.7544, Training Accuracy: 74.30%, Validation Accuracy: 97.70%

Epoch 2/10, Step 100/600, Loss: 0.3705
Epoch 2/10, Step 200/600, Loss: 0.3620
Epoch 2/10, Step 300/600, Loss: 0.2650
Epoch 2/10, Step 400/600, Loss: 0.3186
Epoch 2/10, Step 500/600, Loss: 0.2915
Epoch 2/10, Step 600/600, Loss: 0.3214
Epoch 2/10, Loss: 0.3620, Training Accuracy: 88.19%, Validation Accuracy: 98.05%

Epoch 3/10, Step 100/600, Loss: 0.2805
Epoch 3/10, Step 200/600, Loss: 0.2241
Epoch 3/10, Step 300/600, Loss: 0.3621
Epoch 3/10, Step 400/600, Loss: 0.2215
Epoch 3/10, Step 500/600, Loss: 0.2648
Epoch 3/10, Step 600/600, Loss: 0.2723
Epoch 3/10, Loss: 0.2998, Training Accuracy: 90.14%, Validation Accuracy: 98.47%

Epoch 4/10, Step 100/600, Loss: 0.3066
Epoch 4/10, S

# My MNIST

In [ ]:
myTransform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure single channel
    transforms.Resize((28, 28)),  # Resize to 28x28 in case of any deviations
    transforms.ToTensor(),  # Convert to tensor with range [0, 1]
    transforms.Normalize((0.1307,), (0.3081,)) # mean and std of MNIST dataset
])

In [17]:
from PIL import Image

def classify_image(model, image_path, transform, device):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device) 

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return predicted.item()  # Return the predicted class as an integer

In [18]:
import os

ao_model.eval()

folder_path = './MyMNIST'  
for i in range(10):  # Assuming images are named 0.png to 9.png
    image_path = os.path.join(folder_path, f"{i}.png")
    prediction = classify_image(ao_model, image_path, myTransform, device)
    print(f"Image {i}.png -> Predicted Label: {prediction}")


Image 0.png -> Predicted Label: 8
Image 1.png -> Predicted Label: 8
Image 2.png -> Predicted Label: 8
Image 3.png -> Predicted Label: 8
Image 4.png -> Predicted Label: 8
Image 5.png -> Predicted Label: 8
Image 6.png -> Predicted Label: 8
Image 7.png -> Predicted Label: 8
Image 8.png -> Predicted Label: 8
Image 9.png -> Predicted Label: 8
